### Scraping tool to get reviews from restaurants. Firstly the function scrapeRestaurantsUrlsAll is used to gether all the restaurant urls based on location. Then we iterate through those urls and collect all the reviews.  

### Keyrðu cellurnar þangað til þú sérð

In [18]:
import requests 
from bs4 import BeautifulSoup
import csv 
from selenium import webdriver
import time
import sys
import os
import argparse
import string
import pickle

In [19]:
# Add urls for all "next" pages
def scrapeRestaurantsUrls(tripURLs):
    urls =[]
    for url in tripURLs:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        results = soup.find('div', class_='_1kXteagE')
        stores = results.find_all('div', class_='wQjYiB7z')
        for store in stores:
            unModifiedUrl = str(store.find('a', href=True)['href'])
            urls.append('https://www.tripadvisor.com'+unModifiedUrl)
    return urls

# Add urls for all "next" pages
def scrapeRestaurantsUrlsAll(url, limit=100):
    store_name = []
    urls = []
    limit_set = 1
    nextPage = True
    while nextPage and limit_set <= limit:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        results = soup.find('div', class_='_1kXteagE')
        stores = results.find_all('div', class_='wQjYiB7z') 
        for store in stores:
            if store.find('a', class_ = '_15_ydu6b').text[0].isdigit(): # skip the ones that er sponsored since they will also come later.
                
                print(store.find('a', class_ = '_15_ydu6b').text)
                unModifiedUrl = str(store.find('a', href=True)['href'])
                urls.append('https://www.tripadvisor.com'+unModifiedUrl)
        limit_set += 1
        #Go to next page if exists
        try:
            print('tried next in finding all')
            unModifiedUrl = str(soup.find('a', class_ = 'nav next rndBtn ui_button primary taLnk',href=True)['href'])
            # print(unModifiedUrl, 'later unmod')
            url = 'https://www.tripadvisor.com' + unModifiedUrl
            # print('new url is ', url)
        except:
            print('no next in finding all')
            nextPage = False

    with open(pathAllRestaurants, 'wb') as f:
        pickle.dump(urls, f)

    print(f'Total restaurant count: {len(urls)}')
    return urls

def scrapeRestaurantInfo(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    storeName = soup.find('h1', class_='_3a1XQ88S').text
    avgRating = soup.find('span', class_='r2Cf69qf').text.strip()
    storeAddress = soup.find('div', class_= '_2vbD36Hr _36TL14Jn').find('span', class_='_2saB_OSe').text.strip()
#     urlAddress = str(soup.find('div', class_ = '_2vbD36Hr _36TL14Jn').find('span').find('a', href=True)['href'])
    nrReviews = soup.find('a', class_='_10Iv7dOs').text.strip().split()[0]
    cousineType = [word.text for  word in soup.find('span', class_='_13OzAOXO _34GKdBMV').find_all('a')]
    nrPos = soup.find('a', class_='_15QfMZ2L').find('b').find('span').text.strip()
    with open(pathToStoreInfo, mode='a', encoding="utf-8") as trip:
        data_writer = csv.writer(trip, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
        data_writer.writerow([storeName, storeAddress, avgRating, nrReviews, cousineType[0], cousineType[1:], nrPos])

In [20]:
# driver = webdriver.Chrome('chromedriver.exe')
# This should be set to the path of the folder holding the python script.
    
def get_reviews(url):
    print(url)
    #if you want to scrape restaurants info

#     scrapeRestaurantInfo(url)

    nextPage = True
    while nextPage:
        #Requests
        driver.get(url)
        time.sleep(1)
        #Click More button
#         more = driver.find_elements_by_xpath("//span[contains(text(),'More')]")
        more = driver.find_elements_by_xpath("//span[@class='taLnk ulBlueLinks'][contains(text(),'More')]")
        # Push all buttons that unclude the "More" option on each review.
        for x in range(0,len(more)):
            try:
                driver.execute_script("arguments[0].click();", more[x])
                time.sleep(3)
            except:
                pass
            
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        #Store name
        storeName = soup.find('h1', class_='_3a1XQ88S').text
        #Reviews
        results = soup.find('div', class_='listContainer hide-more-mobile')
        try:
            reviews = results.find_all('div', class_='prw_rup prw_reviews_review_resp')
        except Exception:
            continue
        #Export to csv
        try:
            with open(pathToReviews, mode='a', encoding="utf-8") as trip_data:
                data_writer = csv.writer(trip_data, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
                for review in reviews:
                    ratingDate = review.find('span', class_='ratingDate').get('title')
                    reviewHeader = review.find('span', class_='noQuotes').text
                    text_review = review.find('p', class_='partial_entry')
                    if len(text_review.contents) > 2:
                        reviewText = str(text_review.contents[0][:-3]) + ' ' + str(text_review.contents[1].text)
                    else:
                        reviewText = text_review.text
                    reviewerUsername = review.find('div', class_='info_text pointer_cursor')
                    reviewerUsername = reviewerUsername.select('div > div')[0].get_text(strip=True)
                    rating = review.find('div', class_='ui_column is-9').findChildren('span')
                    rating = str(rating[0]).split('_')[3].split('0')[0]
                    data_writer.writerow([storeName, reviewerUsername, ratingDate, reviewHeader, reviewText, rating])
        except:
            pass

        #Go to next page if exists
        try:
            unModifiedUrl = str(soup.find('div', class_ = 'prw_rup prw_common_responsive_pagination').find('a', class_='nav next ui_button primary', href=True).get('href'))
            # unModifiedUrl = str(soup.find('a', class_ = 'nav next ui_button primary',href=True)['href'])
            url = 'https://www.tripadvisor.com' + unModifiedUrl
        except:
            nextPage = False


## INFO

Þú getur valið hversu mikið þú tekur í keyrslunni en ég held að 300 ætti alveg að vera fínt. 

Það er hægt að copy-a þessu notebook og keyra 2 samtímis þá breytiru bara 0:150 og 150:300 fyrir hvoru keyrsluna.

Ég sé svo um að eyða út fyrstu 300 línunum og skoða svo þau url sem voru með vesen "Save-a url með vesen". Þú mátt runna þá cellu einnig svo vandræðaslóðirnar séu vistaðar.

In [21]:
# Create new reviews.csv path for run
pathToReviews = "reviews8_kristin.csv"

with open(pathToReviews, mode='a', encoding="utf-8") as trip_data:
    data_writer = csv.writer(trip_data, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
    data_writer.writerow(['storeName', 'reviewerUsername', 'ratingDate', 'reviewHeader','reviewText', 'rating'])

In [ ]:
# Þetta er aðalskjalið sem heldur utan um þá linka sem eru eftir.
with open("urls_left.txt", "rb") as f:   # Unpickling
    urls_left = pickle.load(f)

driver_path = f'{os.getcwd()}/chromedriver' # Get driver to run selenium
driver = webdriver.Chrome(driver_path)

finished = []
bad_url = []
rest_300 = urls_left[1000:1500]
i = 0
for url in rest_300:
    try:
        get_reviews(url)
        finished.append(url)
        print(i)
        i+=1
    except:
        bad_url.append(url)
        i+=1

https://www.tripadvisor.com/Restaurant_Review-g189541-d4680259-Reviews-Cafe_World-Copenhagen_Zealand.html
0
https://www.tripadvisor.com/Restaurant_Review-g189541-d6374877-Reviews-PS_Bar_Grill-Copenhagen_Zealand.html
1
https://www.tripadvisor.com/Restaurant_Review-g189541-d14097964-Reviews-Klai_s-Copenhagen_Zealand.html
2
https://www.tripadvisor.com/Restaurant_Review-g189541-d15056025-Reviews-The_Grid-Copenhagen_Zealand.html
3
https://www.tripadvisor.com/Restaurant_Review-g189541-d7972350-Reviews-Rohan_Indian_Restaurant-Copenhagen_Zealand.html
4
https://www.tripadvisor.com/Restaurant_Review-g189541-d21225531-Reviews-RALPHs_Mad_Vin-Copenhagen_Zealand.html
5
https://www.tripadvisor.com/Restaurant_Review-g189541-d21042692-Reviews-Suay-Copenhagen_Zealand.html
6
https://www.tripadvisor.com/Restaurant_Review-g189541-d15335030-Reviews-OliOli-Copenhagen_Zealand.html
7
https://www.tripadvisor.com/Restaurant_Review-g189541-d13861742-Reviews-Letz_Sushi_Valby-Copenhagen_Zealand.html
8
https://www.t

71
https://www.tripadvisor.com/Restaurant_Review-g189541-d1536331-Reviews-Tasty_Fresh_Food-Copenhagen_Zealand.html
72
https://www.tripadvisor.com/Restaurant_Review-g189541-d4332603-Reviews-Tapas_G-Copenhagen_Zealand.html
73
https://www.tripadvisor.com/Restaurant_Review-g189541-d14078663-Reviews-Mokkariet_Odensegade-Copenhagen_Zealand.html
74
https://www.tripadvisor.com/Restaurant_Review-g189541-d3440465-Reviews-Restaurant_Napa-Copenhagen_Zealand.html
75
https://www.tripadvisor.com/Restaurant_Review-g189541-d4697027-Reviews-Min_nye_kaffebar-Copenhagen_Zealand.html
76
https://www.tripadvisor.com/Restaurant_Review-g189541-d8581303-Reviews-Pizza_Palads-Copenhagen_Zealand.html
77
https://www.tripadvisor.com/Restaurant_Review-g189541-d10990315-Reviews-Retreat-Copenhagen_Zealand.html
78
https://www.tripadvisor.com/Restaurant_Review-g189541-d12935892-Reviews-Kaffekilden-Copenhagen_Zealand.html
79
https://www.tripadvisor.com/Restaurant_Review-g189541-d800992-Reviews-Fedtekaelderen-Copenhagen_Ze

143
https://www.tripadvisor.com/Restaurant_Review-g189541-d19885212-Reviews-Muums_Juice-Copenhagen_Zealand.html
144
https://www.tripadvisor.com/Restaurant_Review-g189541-d13143861-Reviews-Espresso_House-Copenhagen_Zealand.html
145
https://www.tripadvisor.com/Restaurant_Review-g189541-d797587-Reviews-Galaxe_Pizza_Bar-Copenhagen_Zealand.html
146
https://www.tripadvisor.com/Restaurant_Review-g806262-d19718114-Reviews-Cafe_Frisk-Frederiksberg_Copenhagen_Zealand.html
147
https://www.tripadvisor.com/Restaurant_Review-g189541-d12540163-Reviews-Milano_Pizzabar-Copenhagen_Zealand.html
148
https://www.tripadvisor.com/Restaurant_Review-g189541-d16183441-Reviews-Carl_s_Madhus-Copenhagen_Zealand.html
149
https://www.tripadvisor.com/Restaurant_Review-g189541-d19510284-Reviews-Meyers_Madhus-Copenhagen_Zealand.html
150
https://www.tripadvisor.com/Restaurant_Review-g189541-d1058006-Reviews-La_Foretta-Copenhagen_Zealand.html
151
https://www.tripadvisor.com/Restaurant_Review-g189541-d19387830-Reviews-Buo

214
https://www.tripadvisor.com/Restaurant_Review-g189541-d11872827-Reviews-Piccola_Cosi-Copenhagen_Zealand.html
215
https://www.tripadvisor.com/Restaurant_Review-g189541-d15335059-Reviews-Sweet_Valentine-Copenhagen_Zealand.html
216
https://www.tripadvisor.com/Restaurant_Review-g806262-d13975842-Reviews-Cafe_Ocean-Frederiksberg_Copenhagen_Zealand.html
217
https://www.tripadvisor.com/Restaurant_Review-g189541-d7208500-Reviews-Nye_Tider-Copenhagen_Zealand.html
218
https://www.tripadvisor.com/Restaurant_Review-g189541-d12342261-Reviews-Neru-Copenhagen_Zealand.html
219
https://www.tripadvisor.com/Restaurant_Review-g189541-d13535143-Reviews-CCR_Sunde_Kokken-Copenhagen_Zealand.html
220
https://www.tripadvisor.com/Restaurant_Review-g189541-d12726909-Reviews-Vi_Viet-Copenhagen_Zealand.html
221
https://www.tripadvisor.com/Restaurant_Review-g189541-d10747034-Reviews-Ehlers_Bageri-Copenhagen_Zealand.html
222
https://www.tripadvisor.com/Restaurant_Review-g189541-d13272807-Reviews-Fovl-Copenhagen_Z

285
https://www.tripadvisor.com/Restaurant_Review-g189541-d8554001-Reviews-Green_Yellow_Restaurant-Copenhagen_Zealand.html
286
https://www.tripadvisor.com/Restaurant_Review-g189541-d19895292-Reviews-Shahi_Pakwaan-Copenhagen_Zealand.html
287
https://www.tripadvisor.com/Restaurant_Review-g189541-d19611500-Reviews-Gasoline_Grill-Copenhagen_Zealand.html
288
https://www.tripadvisor.com/Restaurant_Review-g189541-d4888776-Reviews-Bedste_kaffebar_i_kbh-Copenhagen_Zealand.html
289
https://www.tripadvisor.com/Restaurant_Review-g189541-d7746372-Reviews-Cafe_Baggarden-Copenhagen_Zealand.html
290
https://www.tripadvisor.com/Restaurant_Review-g189541-d12334550-Reviews-Roma_Pizzeria_Grill-Copenhagen_Zealand.html
291
https://www.tripadvisor.com/Restaurant_Review-g189541-d7018364-Reviews-Suyiyaki_Japansk_Restaurant-Copenhagen_Zealand.html
292
https://www.tripadvisor.com/Restaurant_Review-g189541-d18716374-Reviews-Luftkastellet-Copenhagen_Zealand.html
293
https://www.tripadvisor.com/Restaurant_Review-g1

356
https://www.tripadvisor.com/Restaurant_Review-g189541-d812455-Reviews-Kyoto_Restaurant-Copenhagen_Zealand.html
357
https://www.tripadvisor.com/Restaurant_Review-g189541-d12442527-Reviews-Cocks_Cows_Amager-Copenhagen_Zealand.html
358
https://www.tripadvisor.com/Restaurant_Review-g189541-d4128804-Reviews-Cafe_La_Vento-Copenhagen_Zealand.html
359
https://www.tripadvisor.com/Restaurant_Review-g189541-d4802168-Reviews-Lounge_V120-Copenhagen_Zealand.html
360
https://www.tripadvisor.com/Restaurant_Review-g189541-d8592660-Reviews-Spicy_Lounge-Copenhagen_Zealand.html
361
https://www.tripadvisor.com/Restaurant_Review-g189541-d14861969-Reviews-Cafe_Staerkodder-Copenhagen_Zealand.html
362
https://www.tripadvisor.com/Restaurant_Review-g189541-d1017453-Reviews-Beduin_Bar-Copenhagen_Zealand.html
363
https://www.tripadvisor.com/Restaurant_Review-g189541-d3444048-Reviews-Brasseriet-Copenhagen_Zealand.html
364
https://www.tripadvisor.com/Restaurant_Review-g189541-d15017012-Reviews-Coffee_Industry-Co

428
https://www.tripadvisor.com/Restaurant_Review-g189541-d15086633-Reviews-Indian_Tandoor_Restaurant-Copenhagen_Zealand.html
429
https://www.tripadvisor.com/Restaurant_Review-g189541-d11961683-Reviews-DFDS_Restaurant_Little_Italy-Copenhagen_Zealand.html
430
https://www.tripadvisor.com/Restaurant_Review-g806262-d1519861-Reviews-M_G_Petersens_Familiehave-Frederiksberg_Copenhagen_Zealand.html
431
https://www.tripadvisor.com/Restaurant_Review-g189541-d1840386-Reviews-Jin_Cheng-Copenhagen_Zealand.html
432
https://www.tripadvisor.com/Restaurant_Review-g189541-d10840107-Reviews-Nubilo_Pizza-Copenhagen_Zealand.html
433
https://www.tripadvisor.com/Restaurant_Review-g189541-d6678526-Reviews-Restaurant_Laura-Copenhagen_Zealand.html
434
https://www.tripadvisor.com/Restaurant_Review-g806262-d12503741-Reviews-Aroii_Thai_Takeaway-Frederiksberg_Copenhagen_Zealand.html
435
https://www.tripadvisor.com/Restaurant_Review-g189541-d17441444-Reviews-Letz_Sushi_Islands_Brygge-Copenhagen_Zealand.html
436
http

## Save-a url með vesen

In [15]:
# For curiosity how many urls didn't get scraped
print(len(bad_url))

0


In [16]:
# write out the urls that are left to scrape
with open('bad_url4.txt', 'wb') as f:
    pickle.dump(bad_url, f)

In [17]:
for row in bad_url:
    print(row)